<img src="../../EnvXGen/images/EnvXGen_logo.png" alt="EnvXGen_logo">

# Crystal Structure Generation Results Analysis

This notebook is designed for post-processing and analysis of crystalline structure generation results.

## Main Capabilities

### Structure Descriptor Computation
* **RDF (Radial Distribution Function)** - radial distribution function
* **ALIGNN** - descriptors based on atomistic linear graph neural network

### Dimensionality Reduction
* **PCA** (Principal Component Analysis)
* **UMAP** (Uniform Manifold Approximation and Projection)

### Analysis and Visualization
* Structure clustering with optimal cluster number determination
* Cosine similarity calculation between generated and original structures
* 2D and 3D visualization of structure projections before and after relaxation
* Energy characteristics analysis

### Optimal Structure Search
* Automatic search for energetically favorable structures in different clusters

The notebook provides a complete pipeline from descriptor computation to creating informative visualizations for analyzing generated crystalline structures.

---

### Downloading modules and functions

In [1]:
import sys
import os

generator_path = os.path.abspath('../../EnvXGen') # change to '../EnvXGen' if you run postprocessing in the results directory
if generator_path not in sys.path:
    sys.path.append(generator_path)

from postprocessing_scripts import *

/home/u7c/yana/EnvXGen/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Download database

In [2]:
database_filename = f'LaH10_relaxation_results_summarized.pkl'

with open(database_filename, 'rb') as database:
    data = pkl.load(database)

### Calculating Descriptors
<b>Here you can choose one of two algorithms — RDF or ALIGNN</b>

As a result, you will get a  `descriptors\` folder containing a subfolder named after the selected algorithm (`RDF\` or `ALIGNN\`), which will contain the following files:

`generated_structures.csv` - descriptors of generated structures  
`POSCAR_init.csv` - descriptors of initial structure  
`relaxed_structures.csv` - descriptors of structures after relaxation  
`relaxed_structures_init_atoms.csv` - descriptors of initial atoms in relaxed structures  
`relaxed_structures_similarities.csv` - similarities of initial atoms in relaxed structures vs POSCAR_init  

In [3]:
calculate_descriptors(descriptor_algorithm='ALIGNN',
                      poscar_init_path='POSCAR_init',  # change to '../POSCAR_init' if you run postprocessing in the results directory
                      database_filename='LaH10_relaxation_results_summarized.pkl',
                      device='cpu',
                      batch_size=32,
                      n_jobs=-1)

Using device: cpu
Using 12 parallel processes
Loading ALIGNN model...
Calculating descriptor for POSCAR_init...
Processing generated structures...
Calculating descriptors for generated_structures.csv...


Processing generated_structures.csv:   0%|          | 0/7 [00:00<?, ?it/s, batch=1/7]

Processing generated_structures.csv: 100%|██████████| 7/7 [00:44<00:00,  6.32s/it, batch=7/7]


ALIGNN descriptors for generated_structures.csv saved successfully

Processing relaxed structures...
Calculating descriptors for relaxed_structures.csv...


Processing relaxed_structures.csv: 100%|██████████| 7/7 [00:44<00:00,  6.42s/it, batch=7/7]


ALIGNN descriptors for relaxed_structures.csv saved successfully

Processing initial atoms in relaxed structures and calculating similarities...


Processing relaxed structures with init indices: 100%|██████████| 7/7 [00:07<00:00,  1.07s/it]

ALIGNN descriptors for initial atoms in relaxed structures saved successfully
Descriptor calculation completed!


### Reducing Descriptors
<b>Here you can choose one of two algorithms — PCA or UMAP</b>

As a result, you will get new files in `descriptors\RDF\` or `descriptors\ALIGNN\` folder:

`generated_structures_PCA.csv` or `generated_structures_UMAP.csv`  
`relaxed_structures_PCA.csv` or `relaxed_structures_UMAP.csv` 

This files will contain all neccessary information for 2D or 3D visualization.  
You may already have precomputed RDF or ALIGNN descriptors, so be sure to specify which type of descriptors you want to reduce.

In [4]:
reducing_descriptors(descriptor_algorithm='ALIGNN',
                     reducer_algorithm='UMAP',
                     poscar_init_path='POSCAR_init', # default path
                    )

Reducing data


<h3>Visualization of Crystal Structures by Descriptors</h3>

<b>This function allows you to build 2D or 3D plots of crystal structures based on their descriptors</b>
<br>

Users can customize the visualization using the <b>following parameters:</b>


<li><b>descriptor</b>: Type of descriptors used to build the plot. Possible values:
  <ul>
    <code>"RDF"</code> – Radial Distribution Function;<br>
    <code>"ALIGNN"</code> – Descriptors obtained from the ALIGNN model.
  </ul>
</li><br>

  <li><b>reducer</b>: Dimensionality reduction algorithm:
    <ul>
      <code>"PCA"</code> – Principal Component Analysis;<br>
      <code>"UMAP"</code> – Uniform Manifold Approximation and Projection.
    </ul>
  </li><br>

  <li><b>dim</b> (int): Target dimension of the projection:
    <ul>
      <code>2</code> – Two-dimensional projection;<br>
      <code>3</code> – Three-dimensional projection.
    </ul><br>
  </li>

  <li><b>enthalpy</b> (bool): Whether to add formation enthalpy of structures as an additional axis.
    <ul>
      If <code>dim = 2</code> and <code>enthalpy = True</code>, the plot will be 3D: X, Y, Enthalpy;<br>
      If <code>dim = 3</code>, adding an additional axis is not possible – the <code>enthalpy</code> parameter is ignored.
    </ul><br>
  </li>

  <li><b>structures_before_relaxation</b> and <b>structures_after_relaxation</b> (bool):
    <ul>
      Specify whether to display structures before and/or after relaxation.</br>
      At least one of these parameters must be set to <code>True</code>. A plot with neither relaxed nor unrelaxed structures is not allowed.</li>
    </ul>
  </li>



In [10]:
df_results = plot_results(database_filename=database_filename,
                          descriptor_algorithm='ALIGNN',
                          reducer_algorithm='UMAP',
                          descriptors_dimensionality=3,
                          include_enthalpy=False,
                          structures_before_relaxation=False,
                          structures_after_relaxation=True,
                          poscar_init_path='POSCAR_init' # default path
                          )

You can also analyze the <code>df_results</code> dataframe, which contains information in the following format:<p>
<code>ID</code> – ID of the structure  
<code>x</code>, <code>y</code> or <code>x</code>, <code>y</code>, <code>z</code> – coordinates, calculated using reducer  
<code>Energy</code> and <code>Volume</code> of the structure  
<code>SG</code> – space group of the structure
<code>SG_symbol</code> – international symbol of space group  
<code>Cosine_similarity</code> – similarity between initial atoms in relaxed structures vs POSCAR_init  
(насколько сильно итоговая структура похожа на исходную структуру)

### Get top structures from clusters

In [11]:
df_top_structures = find_different_optimal_structures(
    database_filename=database_filename,
    descriptor_algorithm='ALIGNN',
    k=5
)

In [12]:
df_top_structures

,ID,cluster,epoch,CalcFold,generated_structure_energy,generated_structure_volume,generated_structure_SG,generated_structure_symbol,generated_structure,relaxed_structure_energy,relaxed_structure_volume,relaxed_structure_SG,relaxed_structure_symbol,relaxed_structure,warnings
0,ID-1,0,0,1,220.325234,122.722186,1,P1,"(Atom('La', [0.0, 0.0, 0.0], index=0), Atom('L...",5.813417,119.195362,225,Fm-3m,"(Atom('La', [4.9943521087714595, 0.06840171639...",NaN
1,ID-103,0,0,103,202.419791,122.722186,1,P1,"(Atom('La', [0.0, 0.0, 0.0], index=0), Atom('L...",5.814813,119.192734,225,Fm-3m,"(Atom('La', [0.0961726007467167, 0.25562796731...",NaN
2,ID-18,0,0,18,200.655675,122.722186,1,P1,"(Atom('La', [0.0, 0.0, 0.0], index=0), Atom('L...",5.816194,119.191620,225,Fm-3m,"(Atom('La', [0.018095140473032317, 0.016868979...",NaN
3,ID-186,0,0,186,361.338695,122.722186,1,P1,"(Atom('La', [0.0, 0.0, 0.0], index=0), Atom('L...",5.816563,119.178744,225,Fm-3m,"(Atom('La', [4.981641480330498, 0.312502731566...",NaN
4,ID-89,0,0,89,136.345303,122.722186,1,P1,"(Atom('La', [0.0, 0.0, 0.0], index=0), Atom('L...",5.817430,119.175826,225,Fm-3m,"(Atom('La', [0.06853043333580784, 0.1871327104...",NaN
5,ID-190,2,0,190,268.013519,122.722186,1,P1,"(Atom('La', [0.0, 0.0, 0.0], index=0), Atom('L...",6.547064,121.038660,8,Cm,"(Atom('La', [0.16395150542837797, 5.0566960875...",NaN
6,ID-65,2,0,65,222.457097,122.722186,1,P1,"(Atom('La', [0.0, 0.0, 0.0], index=0), Atom('L...",6.743019,120.011979,8,Cm,"(Atom('La', [5.032340462770398, 0.235063366777...",NaN
7,ID-197,2,0,197,178.883768,122.722186,1,P1,"(Atom('La', [0.0, 0.0, 0.0], index=0), Atom('L...",6.743654,119.981427,8,Cm,"(Atom('La', [4.870621341140371, 5.075314700954...",NaN
8,ID-180,2,0,180,144.619402,122.722186,1,P1,"(Atom('La', [0.0, 0.0, 0.0], index=0), Atom('L...",6.744255,119.974780,8,Cm,"(Atom('La', [5.042178547192845, 4.865657368589...",NaN
9,ID-116,2,0,116,247.988944,122.722186,1,P1,"(Atom('La', [0.0, 0.0, 0.0], index=0), Atom('L...",6.744336,120.008461,8,Cm,"(Atom('La', [0.35213297065854304, 4.5745297906...",NaN
